In [1]:
!apt-get update
!apt-get install -y build-essential
!pip install pybind11
!python setup.py build_ext --inplace

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [2]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
import gymnasium as gym
from collections import deque
import random
import copy
from tqdm import tqdm
import Rocket as rck
import math
#import ReplayMemory as rpm


device="cuda" if torch.cuda.is_available() else "gpu"


In [3]:
class Red(nn.Module):

    def __init__(self,il=4,hl=10,ol=2):

        super().__init__()

        self.network=nn.Sequential(nn.Linear(il,hl),
                                  nn.ReLU(),
                                  nn.Linear(hl,hl),
                                  nn.ReLU(),
                                  nn.Linear(hl,ol))
    def forward(self,x):
        return self.network(x)


In [4]:


class ReplayMemory():

    def __init__(self,D=10000):
        self.deque=deque(maxlen=D)
        self.deque_shuffle=deque(maxlen=D)

    def append(self,x):
        self.deque.append(x)

    def minibatch(self,size=32):

        if(len(self.deque_shuffle)==0):
            self.deque_shuffle=copy.deepcopy(self.deque)
            random.shuffle(self.deque_shuffle)


        while(len(self.deque_shuffle)>0):
            ls=[]


            cont=0

            while cont<size and len(self.deque_shuffle)>0:
                ls.append(self.deque_shuffle[0])
                self.deque_shuffle.popleft()
                cont+=1


            yield np.array(ls)

    def reset(self,D=10000):
        self.deque=deque(maxlen=D)
        self.deque_shuffle=deque(maxlen=D)



In [5]:
class WrapAgent():

    def __init__(self,epsilon=0.7,decay=10):
        self.epsilon=epsilon
        self.decay=decay

        self.rng=np.random.default_rng(234343)


    def choose_action(self,env,state,repetition,Qnet,epsilon_o,step_epsilon,cont_epsilon):

        rand=self.rng.uniform(0,1)
        if(rand<(epsilon-step_epsilon*cont_epsilon) or rand<0.01):
            return env.sample()
        else:
            with torch.inference_mode():

                return torch.argmax(Qnet(torch.from_numpy(state).to(device).unsqueeze(dim=0).type(torch.float32))).detach().to("cpu").numpy()

In [6]:
gamma=0.9999
epsilon=1
decay=100
step_epsilon=0.001
cont_epsilon=0
tau=0.05
batch_size=128

start_train=100

max_steps=100
total_repetitions=100
episodes=10000

state_length=9

rp_len=15000

rp=ReplayMemory(rp_len)

agent=WrapAgent(epsilon,decay)


Qnet=Red(state_length,90,4).to(device)
target=Red(state_length,90,4).to(device)

target.load_state_dict(Qnet.state_dict())

optimizer=torch.optim.SGD(Qnet.parameters(),lr=3e-4)
loss=nn.MSELoss()
rng=np.random.default_rng(33234)
env=rck.Rocket()




In [7]:
rew=[]
t_steps=[]
error_t=[]



for i in range(episodes):

    #replay memory (s,r,a,s')



    with torch.inference_mode():
        reward=0
        time_steps=0

        #empiezo un episodio
        state=env.reset()

        for j in range(max_steps):

            action=agent.choose_action(env,state,i,Qnet,epsilon,step_epsilon,cont_epsilon)

            new=env.step(action)


            state_n=new[0:state_length]
            r=new[state_length]
            done=new[-1]

            reward+=r
            time_steps+=1


            if(done==0):
                rp.append(
                    np.hstack((state,r,np.array(action),state_n,done))
                )


                break
            else:

                rp.append(
                    np.hstack((state,r,np.array(action),state_n,done))
                )

                state=state_n



    rew.append(reward)
    t_steps.append(time_steps)

    error=0


    if(i>=start_train):

        for batch in rp.minibatch(batch_size):

            Qnet.train()

            #Estimo el q-value del estado con la red
            state=torch.from_numpy(batch[:,:state_length]).type(torch.float32).to(device)




            finish_states=torch.from_numpy(batch[:,-1]).type(torch.float32).unsqueeze(dim=0).to(device)
            sample_space=np.arange(4)


            actions=torch.vstack(
                tuple(
                    torch.from_numpy(sample_space==action_selected) for action_selected in batch[:,state_length+1]
                )
            ).to(device)



            q_values_qnet=Qnet(state)[actions].unsqueeze(dim=0)




            #Estimo el q_value del estado usando la ecuación de Bellman y la target net
            state_n=torch.from_numpy(batch[:,state_length+2:-1]).type(torch.float32).to(device)
            reward=torch.from_numpy(batch[:,state_length+1]).unsqueeze(dim=0).type(torch.float32).to(device)





            q_values_target=reward+gamma*torch.max(target(state_n),dim=1)[0]*finish_states




            err_train=loss(q_values_target,q_values_qnet)

            error+=(err_train.detach().to("cpu").numpy())


            optimizer.zero_grad()
            err_train.backward()

            optimizer.step()

            Qnet.eval()


        error_t.append(error)

        print(f"Process, {i*100/episodes:03.2f} %,  Episode {i:04d}, Steps {t_steps[-1]:03.0f}, Reward {rew[-1]:03.0f}, Error {error_t[-1]:09.5f}, Epsilon {epsilon-step_epsilon*cont_epsilon if epsilon-step_epsilon*cont_epsilon>0.01 else 0.01:03.3f}")

        if(i%1==0):
            #target.load_state_dict(Qnet.state_dict())

            Qnet_dict=Qnet.state_dict()
            target_dict=target.state_dict()

            for entrada in target_dict.keys():
                target_dict[entrada]=tau*Qnet_dict[entrada]+(1-tau)*target_dict[entrada]

            target.load_state_dict(target_dict)
            torch.save(Qnet.state_dict(),f="net.pth")

        cont_epsilon+=1

Process, 1.00 %,  Episode 0100, Steps 037, Reward 487, Error 067.41627, Epsilon 1.000
Process, 1.01 %,  Episode 0101, Steps 026, Reward -448, Error 064.65163, Epsilon 0.999
Process, 1.02 %,  Episode 0102, Steps 015, Reward -74, Error 062.39238, Epsilon 0.998
Process, 1.03 %,  Episode 0103, Steps 022, Reward 1036, Error 063.52006, Epsilon 0.997
Process, 1.04 %,  Episode 0104, Steps 022, Reward 198, Error 061.51291, Epsilon 0.996
Process, 1.05 %,  Episode 0105, Steps 013, Reward 032, Error 059.69887, Epsilon 0.995
Process, 1.06 %,  Episode 0106, Steps 024, Reward 1297, Error 058.08820, Epsilon 0.994
Process, 1.07 %,  Episode 0107, Steps 023, Reward 892, Error 056.21764, Epsilon 0.993
Process, 1.08 %,  Episode 0108, Steps 015, Reward 030, Error 055.00501, Epsilon 0.992
Process, 1.09 %,  Episode 0109, Steps 019, Reward 303, Error 053.63700, Epsilon 0.991
Process, 1.10 %,  Episode 0110, Steps 032, Reward -489, Error 054.85204, Epsilon 0.990
Process, 1.11 %,  Episode 0111, Steps 015, Reward 

KeyboardInterrupt: 

In [ ]:
torch.save(Qnet.state_dict(),f="net.pth")

In [ ]:
plt.figure(1,figsize=(10,6))

plt.plot(np.arange(len(error_t)),error_t)

In [ ]:
plt.figure(1,figsize=(10,6))

plt.hist(t_steps,bins=100)